# ChIP-seq QC, Alignment and Peak calling

All of these commands can be run in a bash terminal by removing the "!". This is just a jupyter magic command that runs commands using the shell and not using python.

## Get the data

### Download

Wget downloads data from a specfied URL. The files are on the cluster but this should simulate downloading public data.

In [1]:
!wget http://userweb.molbiol.ox.ac.uk/public/asmith/public/chipseq_tutorial/rs411_h3k27ac_chipseq.tar

--2021-03-11 12:24:06--  http://userweb.molbiol.ox.ac.uk/public/asmith/public/chipseq_tutorial/rs411_h3k27ac_chipseq.tar
Resolving userweb.molbiol.ox.ac.uk (userweb.molbiol.ox.ac.uk)... 163.1.16.73
Connecting to userweb.molbiol.ox.ac.uk (userweb.molbiol.ox.ac.uk)|163.1.16.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2296668160 (2.1G) [application/x-tar]
Saving to: ‘rs411_h3k27ac_chipseq.tar.1’

100%[====================================>] 2,296,668,160 90.3MB/s   in 24s    

2021-03-11 12:24:30 (90.3 MB/s) - ‘rs411_h3k27ac_chipseq.tar.1’ saved [2296668160/2296668160]



### Extract

Data is in a TAR archive.These are often gzip compressed to preserve space but I've not done that on this occasion.

In [3]:
!tar -xvf rs411_h3k27ac_chipseq.tar

RS411-histone_H3K27ac_1.fastq.gz
RS411-histone_H3K27ac_2.fastq.gz
RS411_input_1.fastq.gz
RS411_input_2.fastq.gz


## Fastq format

The sequence data is in fastq format i.e.

@IDENTIFIER\
SEQUENCE\
+\
QUALITY SCORES


As this is Paired-End sequencing, Read 1 is present in RS411_input_1.fastq.gz and Read 2 is in RS411_input_2.fastq.gz.

The files are too large to open with a standard text editor so the best way to visualise these is with bash commands.

```zcat``` opens a gzip compressed file.
```|``` Pipes the input from one command to the input of  next.
```head``` prints the first n lines.


In [5]:
!zcat RS411_input_1.fastq.gz | head -n 4

@NB502048:400:HFM25BGXC:1:11101:18003:1044 1:N:0:GATCAG
AGCCANATAATTATTAAAAATACTATATAAATTTACCTTC
+
<AAA/#/EEEEEAAEEEEEEEEEEEEEEEAEEEE/E/E/E

gzip: stdout: Broken pipe


## Stage 1: QC (Fastqc)

```mkdir``` makes a new directory (folder)

```fastqc``` analyses the quality of fastq files and generates HTML outputs.  Flags: -q = quiet, -t = number of threads, --outdir = directory to store output

Ideally would be best to do this for all fastq files and then use multiqc to merge the results together to produce an easier HTML file. In the interest of time will just do one file.

In [3]:
!mkdir fastqc
!fastqc -q -t 8 RS411_input_1.fastq.gz --outdir fastqc

## Stage 2: Trimming

```trim_galore``` removes sequencing adapters from reads. It is unusual to find them in ChIP-seq but it doesn't hurt to check. Flags: --cores = Number of threads --paired = process as paired end reads -o = output directory

In [7]:
!trim_galore --cores 4 --paired -o trimmed RS411_input_1.fastq.gz RS411_input_2.fastq.gz
!trim_galore --cores 4 --paired -o trimmed RS411-histone_H3K27ac_1.fastq.gz RS411-histone_H3K27ac_2.fastq.gz

Path to Cutadapt set as: 'cutadapt' (default)
Cutadapt seems to be working fine (tested command 'cutadapt --version')
Cutadapt version: 1.18
Could not detect version of Python used by Cutadapt from the first line of Cutadapt (but found this: >>>#!/bin/sh<<<)
Letting the (modified) Cutadapt deal with the Python version instead
pigz 2.5
Parallel gzip (pigz) detected. Proceeding with multicore (de)compression using 4 cores

No quality encoding type selected. Assuming that the data provided uses Sanger encoded Phred scores (default)

Output will be written into the directory: /t1-data/user/asmith/Projects/chipseq_tutorial/trimmed/
pigz: abort: write error on <stdout> (Broken pipe)


AUTO-DETECTING ADAPTER TYPE
Attempting to auto-detect adapter type from the first 1 million sequences of the first file (>> RS411_input_1.fastq.gz <<)

Found perfect matches for the following adapter sequences:
Adapter type	Count	Sequence	Sequences analysed	Percentage
Nextera	1	CTGTCTCTTATA	1000000	0.00
smallRN

## Stage 3: Alignment

Next we need to align the reads to the genome. RS4;11 cells are a human MLL-AF4 leukaemia cell line so we'll allign to the human genome. As our group typically works on the hg19 genome build, we will use this one.

```bowtie2``` is the aligner that we will use. This is the one most commonly used for ChIP-seq. Flags: -x = Path to indicies -p = threads. -1 FQ1 -2 FQ2\
```samtools view``` converts BAM <-> SAM. The output of bowtie2 is a SAM file, to reduce storage space, SAM files are converted to a binary BAM format\
```samtools sort``` sorts a bam file\
```samtool index``` creates an index for the bam file. Required for some downstream programs

In [2]:
!mkdir aligned

mkdir: cannot create directory ‘aligned’: File exists


### Align Input

In [6]:
!bowtie2\
--very-fast\
-x /databank/igenomes/Homo_sapiens/UCSC/hg19/Sequence/Bowtie2Index/genome\
-p 8\
-1 trimmed/RS411_input_1_val_1.fq.gz\
-2 trimmed/RS411_input_2_val_2.fq.gz |\
samtools view -b > aligned/RS411_input.bam

10465490 reads; of these:
  10465490 (100.00%) were paired; of these:
    2388171 (22.82%) aligned concordantly 0 times
    6426475 (61.41%) aligned concordantly exactly 1 time
    1650844 (15.77%) aligned concordantly >1 times
    ----
    2388171 pairs aligned concordantly 0 times; of these:
      609927 (25.54%) aligned discordantly 1 time
    ----
    1778244 pairs aligned 0 times concordantly or discordantly; of these:
      3556488 mates make up the pairs; of these:
        1428632 (40.17%) aligned 0 times
        1025548 (28.84%) aligned exactly 1 time
        1102308 (30.99%) aligned >1 times
93.17% overall alignment rate


### Align ChIP

In [7]:
!bowtie2\
--very-fast\
-x /databank/igenomes/Homo_sapiens/UCSC/hg19/Sequence/Bowtie2Index/genome\
-p 8\
-1 trimmed/RS411-histone_H3K27ac_1_val_1.fq.gz\
-2 trimmed/RS411-histone_H3K27ac_2_val_2.fq.gz |\
samtools view -b > aligned/RS411_H3K27ac.bam

25893257 reads; of these:
  25893257 (100.00%) were paired; of these:
    1092869 (4.22%) aligned concordantly 0 times
    21787681 (84.14%) aligned concordantly exactly 1 time
    3012707 (11.64%) aligned concordantly >1 times
    ----
    1092869 pairs aligned concordantly 0 times; of these:
      167280 (15.31%) aligned discordantly 1 time
    ----
    925589 pairs aligned 0 times concordantly or discordantly; of these:
      1851178 mates make up the pairs; of these:
        1145560 (61.88%) aligned 0 times
        292261 (15.79%) aligned exactly 1 time
        413357 (22.33%) aligned >1 times
97.79% overall alignment rate


### Look at alignment

[BAM](https://support.illumina.com/help/BS_App_RNASeq_Alignment_OLH_1000000006112/Content/Source/Informatics/BAM-Format.htm) files are binary and therefore can't be opened with a regular text editor. Samtools view converts BAM files into a human readable SAM output

* Header—Contains information about the entire file, such as sample name, sample length, and alignment method. Alignments in the alignments section are associated with specific information in the header section.
* Alignments—Contains read name, read sequence, read quality, alignment information, and custom tags. The read name includes the chromosome, start coordinate, alignment quality, and the match descriptor string.

* The alignments section includes the following information for each or read pair:\
	▶ 	RG: Read group, which indicates the number of reads for a specific sample.\
	▶ 	BC: Barcode tag, which indicates the demultiplexed sample ID associated with the read.\
	▶ 	SM: Single-end alignment quality.\
	▶ 	AS: Paired-end alignment quality.\
	▶ 	NM: Edit distance tag, which records the Levenshtein distance between the read and the reference.\
	▶ 	XN: Amplicon name tag, which records the amplicon tile ID associated with the read.

### Header

In [11]:
!samtools view -H aligned/RS411_input.bam

@HD	VN:1.0	SO:unsorted
@SQ	SN:chrM	LN:16571
@SQ	SN:chr1	LN:249250621
@SQ	SN:chr2	LN:243199373
@SQ	SN:chr3	LN:198022430
@SQ	SN:chr4	LN:191154276
@SQ	SN:chr5	LN:180915260
@SQ	SN:chr6	LN:171115067
@SQ	SN:chr7	LN:159138663
@SQ	SN:chr8	LN:146364022
@SQ	SN:chr9	LN:141213431
@SQ	SN:chr10	LN:135534747
@SQ	SN:chr11	LN:135006516
@SQ	SN:chr12	LN:133851895
@SQ	SN:chr13	LN:115169878
@SQ	SN:chr14	LN:107349540
@SQ	SN:chr15	LN:102531392
@SQ	SN:chr16	LN:90354753
@SQ	SN:chr17	LN:81195210
@SQ	SN:chr18	LN:78077248
@SQ	SN:chr19	LN:59128983
@SQ	SN:chr20	LN:63025520
@SQ	SN:chr21	LN:48129895
@SQ	SN:chr22	LN:51304566
@SQ	SN:chrX	LN:155270560
@SQ	SN:chrY	LN:59373566
@PG	ID:bowtie2	PN:bowtie2	VN:	CL:"/t1-data/user/asmith/Software/miniconda3/envs/cc/bin/bowtie2-align-s --wrapper basic-0 --very-fast -x /databank/igenomes/Homo_sapiens/UCSC/hg19/Sequence/Bowtie2Index/genome -p 8 -1 trimmed/RS411_input_1_val_1.fq.gz -2 trimmed/RS411_input_2_val_2.fq.gz"


### Alignments

In [12]:
!samtools view aligned/RS411_H3K27ac.bam | head -n 5

NB502048:370:HFM5FBGXC:1:11101:13327:1155	99	chr8	140132068	42	39M	=	140132246	214	AATCGTCGTGAATAGAAAATACATTTCAGCTGGTATCTT	AAAAAEEEEE6EEAEEEEEAEAAEEEEEEAAEEAEAEEE	AS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:39	YS:i:0	YT:Z:CP
NB502048:370:HFM5FBGXC:1:11101:13327:1155	147	chr8	140132246	42	36M	=	140132068	-214	GAGACTTTTTGGGAAGAGCCTTCTTGCCATGGTGCA	EEEEEE6EEEEEEEEEEEEEEEEEEEEEEEEAAAAA	AS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:36	YS:i:0	YT:Z:CP
NB502048:370:HFM5FBGXC:1:11101:25576:1156	77	*	0	0	*	*	0	0	TTAATCAAGCTGCCAAGCAATACTGCAGTCTATAAATTTT	AA/AA6EEEE/EAA/E6/E6/A/E6/6/E6E/666///EE	YT:Z:UP
NB502048:370:HFM5FBGXC:1:11101:25576:1156	141	*	0	0	*	*	0	0	ACTCCCATGACACCAGACAACAGCCATCACTCCACAACA	//A/AEEA/EE//EA/E////E/AE/EE/E/A/A///EE	YT:Z:UP
NB502048:370:HFM5FBGXC:1:11101:18558:1158	83	chr12	131692813	1	40M	=	131692707	-146	CCCTGGGGTGTGGGTTCTCCATTTTCAGTGGATTCGATGG	EEEEEEEEEEEAEE6EEEEEEEEEEEEEEEEEEEEAAAAA	AS:i:0	XS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:40	YS:i:0	YT:Z:CP
samtools view

### Sort BAM files

Required for indexing 

In [13]:
!samtools sort -o aligned/RS411_input_sorted.bam aligned/RS411_input.bam -@ 8
!samtools sort -o aligned/RS411_H3K27ac_sorted.bam aligned/RS411_H3K27ac.bam -@ 8

[bam_sort_core] merging from 0 files and 8 in-memory blocks...
[bam_sort_core] merging from 8 files and 8 in-memory blocks...


### Index BAM files

Required for fast alignment extraction. Used by several downstream programs. 

In [14]:
!samtools index aligned/RS411_input_sorted.bam
!samtools index aligned/RS411_H3K27ac_sorted.bam

## Step 4: Generate a BigWig

To visualise ChIP-seq tracks on a genome browser BAM files have to be converted into the correct format. Effectively these formats list genomic coordinates and provide a score that can be interpreted by the genome browser. The most commonly used is the BigWig which is a binary version of the [wiggle](https://genome.ucsc.edu/goldenPath/help/wiggle.html) format.

Wiggle format mock example:

variableStep chrom=chr2\
300701 12.5\
300702 12.5\
300703 12.5\
300704 12.5\
300705 12.5

Bigwigs can also be generated from [bedgraph](https://genome.ucsc.edu/goldenPath/help/bedgraph.html) format.

Bedgraph mock example:\
chr1    1000    2000    10\
chr1    5000    6000    10

There are multiple methods to generate bigwig files from BAM files the two methods I've used are deeptools and HOMER (bam files need converting to a HOMER TagDirectory first).


```bamCoverage``` from the deeptools suite is potentially the easiest method for converting sorted and index BAM files to bigwig




In [15]:
!mkdir bigwigs

mkdir: cannot create directory ‘bigwigs’: File exists


In [16]:
!bamCoverage -b aligned/RS411_input_sorted.bam -p 8 -o bigwigs/RS411_input.bigWig --effectiveGenomeSize 2864785220 --normalizeUsing RPGC --binSize 50 --smoothLength 100 --extendReads
!bamCoverage -b aligned/RS411_H3K27ac_sorted.bam -p 8 -o bigwigs/RS411_H3K27ac.bigWig --effectiveGenomeSize 2864785220 --normalizeUsing RPGC --binSize 50 --smoothLength 100 --extendReads

/bin/bash: bamCoverage: command not found
/bin/bash: bamCoverage: command not found


## Step 5: Call peaks

Need to identify regions with signal enrichment. 

```MACS2``` is an older but very widely used peak caller.

In [18]:
!mkdir peaks

In [22]:
!macs2\
callpeak\
-t aligned/RS411_H3K27ac_sorted.bam\
-c aligned/RS411_input_sorted.bam\
-g hs\
-f BAMPE\
-n RS411_H3K27ac\
--outdir peaks

/bin/bash: macs2: command not found


# Step 6: Visualisation

There are several methods to visualise both the peaks (bed format) and BigWigs that you have generated, two of the most commonly used are:

1. [IGV](https://software.broadinstitute.org/software/igv/) - Locally installed and faster than UCSC but less easy to share data/ make publication quality figures.
2. [UCSC Genome browser](http://genome-euro.ucsc.edu/) - Web based but need to have a file server to visualise the files (the cluster is able to do this.)


## IGV

IGV should be pre-installed on the cluster but if you want a more up to date version, feel free to run:

```
conda install igv
```

To start IGV run:

```
igv
```

I find the best way to use the browser is to drag and drop files using the file browser into IGV. Alternatively you can search for files to load using the menu.

I can go through this in much more detail if people are interested.


## UCSC

### Move files to an accessible location

In order for UCSC to access the files you have generated, these need to be moved to the public directory on the server.

The best method is to symlink these files to the public directory.

In [6]:
# Get current path
!pwd

/t1-data/user/asmith/Projects/chipseq_tutorial


In [ ]:
!ln -s PWD_OUTPUT/bigwigs/RS411_input.bigWig /public/USERNAME/
!ln -s PWD_OUTPUT/bigwigs/RS411_H3K27ac.bigWig /public/USERNAME/

### Visualisation

Navigate to: http://genome-euro.ucsc.edu/cgi-bin/hgCustom


Fill in the custom track url replacing the bits that I've put in upper case. If it isn't clear in the image, the url is:

http://userweb.molbiol.ox.ac.uk/public/USERNAME/PATH_TO_BIGWIG


![UCSC custom track](https://github.com/alsmith151/chipseq_tutorial/blob/main/ucsc_custom_track.png)

